In [26]:
import numpy as np
import pandas as pd
import copy
import collections
from scipy import linalg
import math
from collections import defaultdict

In [27]:
#data
# index[0]:user_id, index[1]:movie_id, index[2]:ratings
df = pd.read_csv('./ml-100k/u.data',sep='\t',header=None)

In [28]:
df.head(3)

,0,1,2,3
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116


In [29]:
#movie list
df_info = pd.read_csv('./ml-100k/u.item',sep='|',header=None, encoding="ISO-8859-1")

In [30]:
df_info.head(2)

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0


In [31]:
movielist = [df_info[1].tolist()[indx]+';'+str(indx+1) for indx in range(len(df_info[1].tolist()))]

In [32]:
movielist

['Toy Story (1995);1',
 'GoldenEye (1995);2',
 'Four Rooms (1995);3',
 'Get Shorty (1995);4',
 'Copycat (1995);5',
 'Shanghai Triad (Yao a yao yao dao waipo qiao) (1995);6',
 'Twelve Monkeys (1995);7',
 'Babe (1995);8',
 'Dead Man Walking (1995);9',
 'Richard III (1995);10',
 'Seven (Se7en) (1995);11',
 'Usual Suspects, The (1995);12',
 'Mighty Aphrodite (1995);13',
 'Postino, Il (1994);14',
 "Mr. Holland's Opus (1995);15",
 'French Twist (Gazon maudit) (1995);16',
 'From Dusk Till Dawn (1996);17',
 'White Balloon, The (1995);18',
 "Antonia's Line (1995);19",
 'Angels and Insects (1995);20',
 'Muppet Treasure Island (1996);21',
 'Braveheart (1995);22',
 'Taxi Driver (1976);23',
 'Rumble in the Bronx (1995);24',
 'Birdcage, The (1996);25',
 'Brothers McMullen, The (1995);26',
 'Bad Boys (1995);27',
 'Apollo 13 (1995);28',
 'Batman Forever (1995);29',
 'Belle de jour (1967);30',
 'Crimson Tide (1995);31',
 'Crumb (1994);32',
 'Desperado (1995);33',
 'Doom Generation, The (1995);34',
 'Fr

In [33]:
nmovies = len(movielist)
nusers = len(df[0].drop_duplicates().tolist())
print("Movie counts: {}, User counts: {}".format(nmovies, nusers))

Movie counts: 1682, User counts: 943


In [34]:
min_ratings = 50
movies_rated  = list(df[1])
counts = collections.Counter(movies_rated)

In [35]:
counts

Counter({242: 117,
         302: 297,
         377: 13,
         51: 81,
         346: 126,
         474: 194,
         265: 227,
         465: 85,
         451: 170,
         86: 150,
         257: 303,
         1014: 98,
         222: 365,
         40: 57,
         29: 114,
         785: 39,
         387: 65,
         274: 190,
         1042: 28,
         1184: 10,
         392: 68,
         486: 64,
         144: 243,
         118: 293,
         1: 452,
         546: 254,
         95: 219,
         768: 52,
         277: 71,
         234: 280,
         246: 124,
         98: 390,
         193: 157,
         88: 213,
         194: 241,
         1081: 8,
         603: 209,
         796: 36,
         32: 81,
         16: 39,
         304: 149,
         979: 35,
         564: 27,
         327: 175,
         201: 89,
         1137: 29,
         241: 128,
         4: 209,
         332: 143,
         100: 508,
         432: 174,
         322: 218,
         181: 507,
         196: 251,
    

In [36]:
dfout = pd.DataFrame(columns=['user']+movielist)

In [37]:
dfout.head(1)

,user,Toy Story (1995);1,GoldenEye (1995);2,Four Rooms (1995);3,Get Shorty (1995);4,Copycat (1995);5,Shanghai Triad (Yao a yao yao dao waipo qiao) (1995);6,Twelve Monkeys (1995);7,Babe (1995);8,Dead Man Walking (1995);9,...,Mirage (1995);1673,Mamma Roma (1962);1674,"Sunchaser, The (1996);1675","War at Home, The (1996);1676",Sweet Nothing (1995);1677,Mat' i syn (1997);1678,B. Monkey (1998);1679,Sliding Doors (1998);1680,You So Crazy (1994);1681,Scream of Stone (Schrei aus Stein) (1991);1682


In [42]:
toremovelist = []
for i in range(1,nusers):
    tmpmovielist = [0 for j in range(nmovies)]
    dftmp =df[df[0]==i]
    for k in dftmp.index:
        if counts[dftmp.loc[k][1]]>= min_ratings:           
           tmpmovielist[dftmp.loc[k][1]-1] = dftmp.loc[k][2]
           
        else:
           toremovelist.append(dftmp.loc[k][1])
            
    dfout.loc[i] = [i]+tmpmovielist

In [43]:
toremovelist=list(set(toremovelist))
dfout.drop(dfout.columns[toremovelist], axis=1, inplace=True)
dfout.to_csv('utilitymatrix.csv',index=None)

In [44]:
dfout.head(5)

,user,Toy Story (1995);1,GoldenEye (1995);2,Four Rooms (1995);3,Get Shorty (1995);4,Copycat (1995);5,Twelve Monkeys (1995);7,Babe (1995);8,Dead Man Walking (1995);9,Richard III (1995);10,...,Cool Runnings (1993);1035,Hamlet (1996);1039,Forget Paris (1995);1041,Multiplicity (1996);1047,She's the One (1996);1048,Koyaanisqatsi (1983);1065,Shallow Grave (1994);1073,Reality Bites (1994);1074,Six Degrees of Separation (1993);1101,Some Kind of Wonderful (1987);1119
1,1,5,3,4,3,3,4,1,5,3,...,0,0,0,0,0,0,0,0,0,0
2,2,4,0,0,0,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,5,4,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 영화 장르 데이터

In [45]:
#matrix movies's content
movieslist = [int(m.split(';')[-1]) for m in dfout.columns[1:]]
moviescats = ['unknown','Action','Adventure','Animation','Children\'s','Comedy','Crime','Documentary',
              'Drama','Fantasy','Film-Noir','Horror','Musical','Mystery',
              'Romance','Sci-Fi','Thriller','War','Western']
dfout_movies =  pd.DataFrame(columns=['movie_id']+moviescats)
startcatsindx = 5
cnt= 0
for m in movieslist:
    dfout_movies.loc[cnt] = [m]+df_info.iloc[m-1][startcatsindx:].tolist()
    cnt +=1 
print(dfout_movies.head())

dfout_movies.to_csv('movies_content.csv',index=None)

  movie_id unknown Action Adventure Animation Children's Comedy Crime  \
0        1       0      0         0         1          1      1     0   
1        2       0      1         1         0          0      0     0   
2        3       0      0         0         0          0      0     0   
3        4       0      1         0         0          0      1     0   
4        5       0      0         0         0          0      0     1   

  Documentary Drama Fantasy Film-Noir Horror Musical Mystery Romance Sci-Fi  \
0           0     0       0         0      0       0       0       0      0   
1           0     0       0         0      0       0       0       0      0   
2           0     0       0         0      0       0       0       0      0   
3           0     1       0         0      0       0       0       0      0   
4           0     1       0         0      0       0       0       0      0   

  Thriller War Western  
0        0   0       0  
1        1   0       0  
2        1 

### 결측치 대체(Imputation)

In [46]:
def imputation(inp,Ri):
    Ri = Ri.astype(float)
    def userav():
        for i in range(len(Ri)):
            Ri[i][Ri[i]==0] = sum(Ri[i])/float(len(Ri[i][Ri[i]>0]))
        return Ri
    def itemav():
        for i in range(len(Ri[0])):
            Ri[:,i][Ri[:,i]==0] = sum(Ri[:,i])/float(len(Ri[:,i][Ri[:,i]>0]))
        return Ri            
    switch = {'useraverage':userav(),'itemaverage':itemav()}
    return switch[inp]

### 유사도 평가

In [47]:
from scipy.stats import pearsonr
from scipy.spatial.distance import cosine 
def sim(x,y,metric='cos'):
    if metric == 'cos':
       return 1.-cosine(x,y)
    else:#correlation
       return pearsonr(x,y)[0]

## 협업 필터링 방법 (Collaborative Filtering Methods)

### 메모리 기반 협업 필터링

* 메모리 기반 형업 필터링은 **사용자 기반 협업 필터링(User-Based Collaborative Filtering)**과 **아이템 기반 협업 필터링(item-Based Collaborative Filtering)**으로 구분할 수 있다.

#### 1. 사용자 기반 협업 필터링

* **u_vec**은 사용자 평점 벡터로 함수 **FindKNeighbours**에서 이 벡터와 가장 비슷한 사용자 $K$를 찾는다. 
* **CalcRating** 함수는 앞의 식을 이용해서 예측 평점을 계산한다. 단, 분포 교정은 하지 않는다.
* 유틸리티 행렬이 데이터가 별로 없어 이웃이 없는 경우 사용자의 평균 평점으로 예측이 이루어지도록 구현되었다.
* 예측 평점이 5를 넘거나 1보다 낮은 경우 각각 5와 1로 조정된다.

In [48]:
def CF_userbased(u_vec,K,data,indxs=False):
    def FindKNeighbours(r,data,K):
        neighs = []
        cnt=0
        for u in range(len(data)):
            if data[u,r]>0 and cnt<K:
               neighs.append(data[u])   
               cnt +=1 
            elif cnt==K:
               break
        return np.array(neighs)
        
    def CalcRating(u_vec,r,neighs):
        rating = 0.
        den = 0.
        for j in range(len(neighs)):
            rating += neighs[j][-1]*float(neighs[j][r]-neighs[j][neighs[j]>0][:-1].mean())
            den += abs(neighs[j][-1])
        if den>0:
            rating = np.round(u_vec[u_vec>0].mean()+(rating/den),0)
        else:
            rating = np.round(u_vec[u_vec>0].mean(),0)
        if rating>5:
            return 5.
        elif rating<1:
            return 1.
        return rating 
    #add similarity col
    data = data.astype(float)
    nrows = len(data)
    ncols = len(data[0])
    data_sim = np.zeros((nrows,ncols+1))
    data_sim[:,:-1] = data
    #calc similarities:
    for u in range(nrows):
        if np.array_equal(data_sim[u,:-1],u_vec)==False: #list(data_sim[u,:-1]) != list(u_vec):
           data_sim[u,ncols] = sim(data_sim[u,:-1],u_vec,'pearson')
        else:
           data_sim[u,ncols] = 0.
    #order by similarity:
    data_sim =data_sim[data_sim[:,ncols].argsort()][::-1]
    #find the K users for each item not rated:
    u_rec = np.zeros(len(u_vec))
    for r in range(ncols):
        if u_vec[r]==0:
           neighs = FindKNeighbours(r,data_sim,K)
           #calc the predicted rating
           u_rec[r] = CalcRating(u_vec,r,neighs)
    if indxs:
            #take out the rated movies
            seenindxs = [indx for indx in range(len(u_vec)) if u_vec[indx]>0]
            u_rec[seenindxs] = -1
            recsvec = np.argsort(u_rec)[::-1][np.argsort(u_rec)>0]
        
            return recsvec    
    return u_rec

#### 2. 아이템 기반 협업 필터링

* 클래스 **CF_itembased**의 생성자는 아이템 유사도 행렬 **simmatrix**를 계산한다.
* 아이템에 대해 사용자 평점이 없는 경우 함수 **CalcRatings**를 통해 예측을 하게 되며, 이 때 함수 내부적으로 **simmatrix**를 사용한다.
* 함수 **GetKSimItemsperUser**는 사용자가 평가하지 않은 아이템과 유사도가 가장 높은 아이템 중 사용자가 과거에 평가했던 $K$개의 아이템을 찾는다.
* **u_vec**는 사용자 평점 벡터로, 유틸리티 행렬의 행 벡터이다. **CalcRatings**은 가중 평균 평점을 계산한다. 이웃을 찾을 수 없는 경우 평점은 아이템의 평균 평점으로 설정한다.

In [49]:
class CF_itembased(object):
    def __init__(self,data):
        #calc item similarities matrix
        nitems = len(data[0])
        self.data = data
        self.simmatrix = np.zeros((nitems,nitems))
        for i in range(nitems):
            for j in range(nitems):
                if j>=i:#triangular matrix
                   self.simmatrix[i,j] = sim(data[:,i],data[:,j])
                else:
                   self.simmatrix[i,j] = self.simmatrix[j,i]

    def GetKSimItemsperUser(self,r,K,u_vec):
        items = np.argsort(self.simmatrix[r])[::-1]
        items = items[items!=r]
        cnt=0
        neighitems = []
        for i in items:
            if u_vec[i]>0 and cnt<K:
               neighitems.append(i)
               cnt+=1
            elif cnt==K:
               break
        return neighitems
        
    def CalcRating(self,r,u_vec,neighitems):
        rating = 0.
        den = 0.
        for i in neighitems:
            rating +=  self.simmatrix[r,i]*u_vec[i]
            den += abs(self.simmatrix[r,i])
        if den>0:
            rating = np.round(rating/den,0)
        else:
            rating = np.round(self.data[:,r][self.data[:,r]>0].mean(),0)
        return rating
        
    def CalcRatings(self,u_vec,K,indxs=False):
        #u_rec = copy.copy(u_vec)
        u_rec = np.zeros(len(u_vec))
        for r in range(len(u_vec)):
            if u_vec[r]==0:
               neighitems = self.GetKSimItemsperUser(r,K,u_vec)
               #calc predicted rating
               u_rec[r] = self.CalcRating(r,u_vec,neighitems)
        if indxs:
            #take out the rated movies
            seenindxs = [indx for indx in range(len(u_vec)) if u_vec[indx]>0]
            u_rec[seenindxs]=-1
            recsvec = np.argsort(u_rec)[::-1][np.argsort(u_rec)>0]
        
            return recsvec
        return u_rec

### 트레이닝 셋과 테스트 셋 분리

In [50]:
def cross_validation(df,k):
    val_num = int(len(df)/float(k))
    print(val_num)
    df_trains = []
    df_vals = []
    for i in range(k):
        start_val = (k-i-1)*val_num
        end_val = start_val+val_num
        df_trains.append(pd.concat([df[:start_val],df[end_val:]]))
        df_vals.append(df[start_val:end_val])

    return df_trains,df_vals

### RMSE 평가

In [51]:
def SE(u_preds,u_vals):
    nratings = len(u_vals)
    se = 0.
    cnt = 0
    for i in range(nratings):
        if u_vals[i]>0:
           se +=  (u_vals[i]-u_preds[i])*(u_vals[i]-u_preds[i])
           cnt += 1
    return se,cnt

### 예측을 위해 U_VEC의 절반은 숨겨서 값을 예측하긴 위한 적업

In [52]:
import random
def HideRandomRatings(u_vec, ratiovals=0.5):
    u_test = np.zeros(len(u_vec))
    u_vals = np.zeros(len(u_vec))
    cnt = 0
    nratings = len(u_vec[u_vec>0])
    for i in range(len(u_vec)):
        if u_vec[i]>0:        
            if bool(random.getrandbits(1)) or cnt>=int(nratings*ratiovals):
                u_test[i]=u_vec[i]
            else:#random choice to hide the rating:
                cnt +=1
                u_vals[i]=u_vec[i]
    return u_test,u_vals

### 1. 데이터 로드

In [53]:
#load data
df = pd.read_csv('utilitymatrix.csv')
print(df.head(4))
df_movies = pd.read_csv('movies_content.csv')
movies = df_movies.values[:,1:]
print('check:::',len(df.columns[1:]),'--',len(df_movies))
movieslist = list(df.columns[1:])
#k-fold cv 5 folds
nfolds = 5
df_trains,df_vals = cross_validation(df,nfolds)

   user  Toy Story (1995);1  GoldenEye (1995);2  Four Rooms (1995);3  \
0     1                   5                   3                    4   
1     2                   4                   0                    0   
2     3                   0                   0                    0   
3     4                   0                   0                    0   

   Get Shorty (1995);4  Copycat (1995);5  Twelve Monkeys (1995);7  \
0                    3                 3                        4   
1                    0                 0                        0   
2                    0                 0                        0   
3                    0                 0                        0   

   Babe (1995);8  Dead Man Walking (1995);9  Richard III (1995);10  ...  \
0              1                          5                      3  ...   
1              0                          0                      2  ...   
2              0                          0                      0  

### 2. 데이터 준비

In [54]:
nmovies = len(df_vals[0].values[:,1:][0])
vals_vecs_folds = []
tests_vecs_folds = []
for i in range(nfolds):
    u_vecs = df_vals[i].values[:,1:]
    vtests = np.empty((0,nmovies),float)
    vvals = np.empty((0,nmovies),float)
    for u_vec in u_vecs:
        u_test,u_vals = HideRandomRatings(u_vec)
        vvals = np.vstack([vvals,u_vals])
        vtests = np.vstack([vtests,u_test])
    vals_vecs_folds.append(vvals)
    tests_vecs_folds.append(vtests)

### 3. 평가

* 아이템 기반 협업

In [83]:
err_itembased = 0.
cnt_itembased = 0

for i in range(nfolds):
    Umatrix = df_trains[i].values[:,1:]
    cfitembased = CF_itembased(Umatrix)

    print('fold:',i+1)
    vec_vals = vals_vecs_folds[i]
    vec_tests = tests_vecs_folds[i]
    
    for j in range(len(vec_vals)):
        u_vals = vec_vals[j]
        u_test = vec_tests[j]
        
        #cf_itembased
        u_preds = cfitembased.CalcRatings(u_test,5)
        e,c =  SE(u_preds,u_vals)
        err_itembased +=e
        cnt_itembased +=c
        
rmse_itembased = np.sqrt(err_itembased/float(cnt_itembased))

print('user_itembased rmse:',rmse_itembased,'--',cnt_itembased)

fold: 1
fold: 2
fold: 3
fold: 4
fold: 5
user_itembased rmse: 1.02814285969 -- 39928


In [84]:
u_preds

array([ 4.,  3.,  4.,  4.,  0.,  0.,  4.,  5.,  3.,  4.,  4.,  3.,  4.,
        3.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  3.,  3.,  4.,  3.,
        4.,  4.,  4.,  0.,  4.,  3.,  3.,  3.,  3.,  3.,  4.,  3.,  4.,
        4.,  3.,  4.,  4.,  4.,  0.,  4.,  3.,  4.,  3.,  3.,  4.,  4.,
        4.,  0.,  3.,  4.,  4.,  4.,  4.,  3.,  4.,  0.,  0.,  0.,  3.,
        3.,  4.,  4.,  3.,  4.,  4.,  0.,  4.,  3.,  3.,  4.,  4.,  3.,
        4.,  4.,  4.,  4.,  3.,  4.,  3.,  4.,  3.,  4.,  4.,  3.,  3.,
        3.,  4.,  4.,  4.,  3.,  0.,  3.,  4.,  4.,  3.,  4.,  0.,  3.,
        4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,
        4.,  4.,  4.,  0.,  4.,  0.,  3.,  0.,  4.,  3.,  4.,  0.,  4.,
        0.,  4.,  0.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  3.,
        4.,  4.,  4.,  4.,  0.,  3.,  4.,  4.,  0.,  0.,  4.,  4.,  3.,
        4.,  4.,  4.,  4.,  4.,  4.,  0.,  4.,  4.,  4.,  4.,  4.,  4.,
        4.,  4.,  4.,  4.,  0.,  4.,  0.,  4.,  3.,  4.,  3.,  0

In [85]:
u_vals

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  5.,  0.,  4.,  0.,
        0.,  0.,  0.,  0.,  0.,  5.,  0.,  0.,  0.,  0.,  0.,  3.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  4.,
        0.,  0.,  0.,  4.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  5.,
        0.,  0.,  0.,  0.,  4.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  5.,  5.,  5.,  0.,  4.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  3.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  5.,
        3.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  4.,  0.,  4.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  5.,  5.,  0.,  0.,  4.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        4.,  0.,  3.,  0.,  0.,  0.,  0.,  0.,  0.,  3.,  3.,  0.,  0.,
        0.,  4.,  0.,  0.,  0.,  0.,  0.,  3.,  0.,  0.,  0.,  0

* 사용자 기반 협업

In [86]:
err_userbased = 0.
cnt_userbased = 0

for i in range(nfolds):
    Umatrix = df_trains[i].values[:,1:]
    
    print('fold:',i+1)
    vec_vals = vals_vecs_folds[i]
    vec_tests = tests_vecs_folds[i]
    
    for j in range(len(vec_vals)):
        u_vals = vec_vals[j]
        u_test = vec_tests[j]
        
        #cf_userbased
        u_preds = CF_userbased(u_test,5,Umatrix)
        e,c =  SE(u_preds,u_vals)
        err_userbased +=e
        cnt_userbased +=c
        
rmse_userbased = np.sqrt(err_userbased/float(cnt_userbased))

print('user_userbased rmse:',rmse_userbased,'--',cnt_userbased)

fold: 1
fold: 2
fold: 3
fold: 4
fold: 5
user_userbased rmse: 1.02015846903 -- 39928


#### Classification Reports

In [55]:
def ClassificationMetrics(vec_vals,vec_recs,likethreshold=3,shortlist=50,ratingsval=False,vec_test=None):
    #convert vals in indxs vec
    indxs_like = [i for i in range(len(vec_vals)) if vec_vals[i]>likethreshold]
    indxs_dislike = [i for i in range(len(vec_vals)) if vec_vals[i]<=likethreshold and vec_vals[i]>0]
    cnt = len(indxs_like)+len(indxs_dislike)
    indxs_rec = []
    if ratingsval:
        #convert ratings into items's list
        #if vec_test==None:
        #   raise 'Error no test vector'
        indxs_rec = [i for i in range(len(vec_recs)) if vec_recs[i]>likethreshold and vec_test[i]<1][:shortlist]
    else:
        #consider only the first slot of recs
        indxs_rec = vec_recs[:shortlist]

    tp = len(set(indxs_rec).intersection(set(indxs_like)))
    fp = len(set(indxs_rec).intersection(set(indxs_dislike)))
    fn = len(set(indxs_like)^(set(indxs_rec).intersection(set(indxs_like))))
    precision = 0.
    if tp+fp>0:
        precision = float(tp)/(tp+fp)
    recall = 0.
    if tp+fn>0:
        recall = float(tp)/(tp+fn)
    f1 = 0.
    if recall+precision >0:
        f1 = 2.*precision*recall/(precision+recall)
    
    return np.array([precision,recall,f1]),cnt

#### 사용자 기반 CF의 Classification Report

In [56]:
tot_measures = np.zeros(3)    
cnt_vals = 0.
#CF memory based
for i in range(nfolds):
    Umatrix = df_trains[i].values[:,1:]
    
    print('fold:',i+1)
    tot_measures_fold = np.zeros(3)
    vals_vecs = vals_vecs_folds[i]
    tests_vecs = tests_vecs_folds[i]
    
    for j in range(len(vals_vecs)):
        u_vals = vals_vecs[j]
        u_test = tests_vecs[j]
        u_preds = CF_userbased(u_test,20,Umatrix)
        
        tmp_measures,cnt_tmp = ClassificationMetrics(u_vals,u_preds,3,50,True,u_test)
        tot_measures_fold +=  tmp_measures
        cnt_vals += cnt_tmp
        
    tot_measures_fold /= float(len(vals_vecs))
    print(tot_measures_fold)
    tot_measures += tot_measures_fold
    
tot_measures /= float(nfolds)

print('/n')
print('precision:',tot_measures[0],' recall:',tot_measures[1],' f1:',tot_measures[2],'---',cnt_vals)

fold: 1
[0.56537734 0.16773659 0.24313414]
fold: 2


KeyboardInterrupt: 

#### 아이템 기반 CF의 Classification Report

In [97]:
tot_measures = np.zeros(3)    
cnt_vals = 0.
#CF memory based
for i in range(nfolds):
    Umatrix = df_trains[i].values[:,1:]
    cfitembased = CF_itembased(Umatrix)
    
    print('fold:',i+1)
    tot_measures_fold = np.zeros(3)
    vals_vecs = vals_vecs_folds[i]
    tests_vecs = tests_vecs_folds[i]
    
    for j in range(len(vals_vecs)):
        u_vals = vals_vecs[j]
        u_test = tests_vecs[j]
        u_preds = cfitembased.CalcRatings(u_test,5)
        
        tmp_measures,cnt_tmp = ClassificationMetrics(u_vals,u_preds,3,50,True,u_test)
        tot_measures_fold +=  tmp_measures
        cnt_vals += cnt_tmp
        
    tot_measures_fold /= float(len(vals_vecs))
    print(tot_measures_fold)
    tot_measures += tot_measures_fold
    
tot_measures /= float(nfolds)

print('/n')
print('precision:',tot_measures[0],' recall:',tot_measures[1],' f1:',tot_measures[2],'---',cnt_vals)

fold: 1


/Users/jason/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:9: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


[ 0.55580163  0.14017202  0.21124015]
fold: 2
[ 0.57972669  0.15345869  0.22856806]
fold: 3
[ 0.55977389  0.13844284  0.21064714]
fold: 4
[ 0.58311225  0.14445215  0.22230857]
fold: 5
[ 0.59793147  0.14984474  0.22632978]
/n
precision: 0.575269184431  recall: 0.145274090414  f1: 0.219818740449 --- 39928.0


# Good Job